In [2]:
import requests
import pandas as pd
import numpy as np
from xml.etree import ElementTree
from collections import defaultdict
import time
from tqdm import tqdm

### Set up API Environment

In [3]:
s = requests.Session() # create session
# Po`st login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'aniketh@stanford.edu', 'password':'dcy*BWEfAm'})

<Response [200]>

In [4]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

### Get data

In [5]:
flows = pd.read_csv("whole_cell_flow_data.csv")
stoichiometry = pd.read_csv("whole_cell_s_matrix (1).csv")
metabolite_names = list(stoichiometry['Unnamed: 0'])
reaction_names = list(stoichiometry.columns[1:])
n_mets = len(metabolite_names)
n_rxns = len(reaction_names)
S = stoichiometry.iloc[:, 1:].to_numpy()

In [6]:
metabolites_processed = {}
for meta in metabolite_names:
    new = meta.split('[', 1)[0]
    metabolites_processed[meta] = new

In [7]:
reactions_processed = {}

for rxn in reaction_names:
    new = rxn
    
    if "(reverse)" in new:
        idx = new.find("(reverse)")
        new = new[:idx - 1]
    
    if "__" in new:
        idx = new.find("__")
        new = new[:idx]
    
    if "-RXN" in new:
        idx = new.find("-RXN")
        new = new[:idx + 4]
    
    reactions_processed[rxn] = new

In [8]:
pathway_dict = {}
notfound_meta = []
curr_time = 0
for meta in tqdm(metabolite_names):
    pathway_dict[meta] = []
    object_id = metabolites_processed[meta]
    request = "https://websvc.biocyc.org/apixml?fn=pathways-of-compound&id=ECOLI:" + object_id
    
    response = s.get(request)
    
    if response.status_code == 200:
        tree = ElementTree.fromstring(response.content)
        r = etree_to_dict(tree)
    else:
        notfound_meta.append((meta, response.status_code))
        continue
    
    try:
        curr = r['ptools-xml']['Pathway']
        if type(curr) is list:
            for val in curr:
                pathway_dict[meta].append(val['@frameid'])
        else:
            pathway_dict[meta].append(curr['@frameid'])
    except KeyError:
        notfound_meta.append((meta, KeyError))

100%|█████████████████████████████████████████| 489/489 [01:29<00:00,  5.45it/s]


In [9]:
notfound_rxn = []
for rxn in tqdm(reaction_names):
    pathway_dict[rxn] = []
    object_id = reactions_processed[rxn]
    request = "https://websvc.biocyc.org/getxml?id=ECOLI:" + object_id
    
    response = s.get(request)
    
    if response.status_code == 200:
        tree = ElementTree.fromstring(response.content)
        r = etree_to_dict(tree)
    else:
        notfound_rxn.append((rxn, response.status_code))
        continue
    
    try:
        curr = r['ptools-xml']['Reaction']['in-pathway']['Pathway']
        if type(curr) is list:
            for val in curr:
                pathway_dict[rxn].append(val['@frameid'])
        else:
            pathway_dict[rxn].append(curr['@frameid'])
    except KeyError:
        notfound_rxn.append((rxn, KeyError))

100%|█████████████████████████████████████████| 470/470 [01:01<00:00,  7.60it/s]


In [11]:
len(notfound_meta + notfound_rxn)

147

In [37]:
count = 0
total = 0

for key in pathway_dict:
    if len(pathway_dict[key]) <= 5:
        count += 1
    total += 1

In [38]:
count

825

In [39]:
total

959

In [40]:
import json

In [41]:
# Save the dictionary to a file using JSON
with open('pathway.json', 'w') as f:
    json.dump(pathway_dict, f)

In [10]:
response = s.get("https://websvc.biocyc.org/getxml?id=ECOLI:PWY-7222")
    
if response.status_code == 200:
    tree = ElementTree.fromstring(response.content)
    r = etree_to_dict(tree)
else:
    notfound_rxn.append((rxn, response.status_code))
    continue

NameError: name 's' is not defined